## Word Embeddings

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#The indices of the words represent the values in the lookup table
word_to_idx = {"I": 0, "love": 1, "eating":2, "and":3, "sleeping":4}
embeddings = nn.Embedding(5, 7)    # 5 words in vocab, embedding size is 7
word_index = torch.tensor([word_to_idx["love"]])
print(word_index)

tensor([1])


In [3]:
love = embeddings(word_index)
print(love)
print(love.shape)

tensor([[-0.8540,  1.5233, -1.0239, -1.1388, -0.0016, -0.8607, -0.2642]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 7])


In [4]:
all_ind = torch.tensor([w for w in range(5)], dtype = torch.long)
all_words = embeddings(all_ind)
print(all_ind)
print(all_words)
print(all_words.shape)

tensor([0, 1, 2, 3, 4])
tensor([[ 1.2555e+00, -1.5690e+00,  6.0701e-01,  1.2811e+00,  3.0316e-01,
          1.2836e+00,  2.4289e-01],
        [-8.5397e-01,  1.5233e+00, -1.0239e+00, -1.1388e+00, -1.6443e-03,
         -8.6073e-01, -2.6416e-01],
        [-5.4758e-01, -7.5587e-01, -2.8377e-01, -8.0554e-01, -1.0463e+00,
         -3.9389e-01, -2.8749e-01],
        [ 2.8509e-01, -8.9369e-01,  4.8759e-01, -3.3821e-02,  2.2692e+00,
          2.0623e+00, -5.0913e-02],
        [-4.0016e-01,  6.7093e-01, -6.9782e-01, -5.2599e-01, -6.5403e-01,
          5.5537e-01,  7.3809e-01]], grad_fn=<EmbeddingBackward0>)
torch.Size([5, 7])


## Exercise: N-Gram Skip Model
### Given a sequence of words, we want to predict the ith word of the sequence: P(w(i)|w(i-1), w(i-2), .....)
#### Source: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#an-example-n-gram-language-modeling  


In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[519.1281599998474, 516.493239402771, 513.8753712177277, 511.27360558509827, 508.6874017715454, 506.11615467071533, 503.55976581573486, 501.0175971984863, 498.48900747299194, 495.9734330177307]
tensor([-0.8846, -0.1527,  1.4066, -0.6703,  0.8545, -3.7965,  1.0284, -0.0567,
        -0.2338, -0.7578], grad_fn=<SelectBackward0>)
